## Preprocessing

In [269]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [270]:
# Drop the non-beneficial ID columns, 'NAME'.
application_df = application_df.drop(columns= ['EIN'])
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [271]:
# Determine the number of unique values in each column.
application_df.nunique().sort_values(ascending=False)

NAME                      19568
ASK_AMT                    8747
CLASSIFICATION               71
APPLICATION_TYPE             17
INCOME_AMT                    9
AFFILIATION                   6
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
SPECIAL_CONSIDERATIONS        2
IS_SUCCESSFUL                 2
dtype: int64

In [272]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts().sort_values(ascending=False)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [273]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T17', 'T15', 'T29', 'T14', 'T25', 'T2', 'T12', 'T13', 'T9'] 

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts().sort_values(ascending=False)

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [274]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts().sort_values(ascending=False)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C5200        1
C2600        1
C4200        1
C2190        1
C2150        1
Name: count, Length: 71, dtype: int64

In [275]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 2].sort_values(ascending=False)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
Name: count, dtype: int64

In [276]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
print(application_df['USE_CASE'].value_counts())
def replacer(column, sign, x):
    value_counts = column.value_counts()
    for cls in value_counts.index:
        if (sign == '<' and value_counts[cls] < x) or (sign == '>' and value_counts[cls] > x):
            column = column.replace(cls, "Other")
    return column

application_df['CLASSIFICATION'] = replacer(application_df['CLASSIFICATION'], '<', 1883)
application_df['NAME'] = replacer(application_df['NAME'], '<', 100)
application_df['USE_CASE'] = replacer(application_df['USE_CASE'], '<', 3)

USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: count, dtype: int64


In [277]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, dummy_na=False, dtype=int)

In [278]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns="IS_SUCCESSFUL").values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [279]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [280]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=45, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

C:\Users\3aama\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_64 (Dense)                     │ (None, 80)                  │           6,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_65 (Dense)                     │ (None, 45)                  │           3,645 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_66 (Dense)                     │ (None, 100)                 │           4,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_67 (Dense)                     │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,346 (56.04 KB)

 Trainable params: 14,346 (56.04 KB)

 Non-trainable params: 0 (0.00 B)

In [281]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [282]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 750us/step - accuracy: 0.7296 - loss: 0.5327
Epoch 2/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 810us/step - accuracy: 0.7577 - loss: 0.4928
Epoch 3/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 729us/step - accuracy: 0.7558 - loss: 0.4941
Epoch 4/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 721us/step - accuracy: 0.7607 - loss: 0.4881
Epoch 5/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 737us/step - accuracy: 0.7604 - loss: 0.4865
Epoch 6/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 736us/step - accuracy: 0.7632 - loss: 0.4823
Epoch 7/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 774us/step - accuracy: 0.7612 - loss: 0.4863
Epoch 8/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 720us/step - accuracy: 0.7635 - loss: 0.4825
Epoch 9/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 724us/step - accuracy: 0.7647 - loss: 0.4803
Epoch 10/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 736us/step - accuracy: 0.7669 - loss: 0.4762


In [283]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 954us/step - accuracy: 0.7536 - loss: 0.4946
Loss: 0.4946429133415222, Accuracy: 0.7535859942436218


In [285]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')